In [1]:
from difflib import SequenceMatcher
import regex as re
from tqdm import tqdm_notebook as tqdm
from collections import Counter

In [2]:
txt_ref_path = "DATA/SOMUN.TN.txt"
txt_trg_path = "DATA/DYBG.TN.txt"

with open(txt_ref_path, 'r', encoding="utf-8") as fl:
    txt_ref_raw = fl.read()

with open(txt_trg_path, 'r', encoding="utf-8") as fl:
    txt_trg_raw = fl.read()

In [3]:
# hanzi 범위
cjk_range = "[\p{Han}]"
cjk_range_re = re.compile( cjk_range, re.UNICODE)

# hanzi 추출
txt_ref = "".join( re.findall( cjk_range_re, txt_ref_raw  ) )
txt_trg = "".join( re.findall( cjk_range_re, txt_trg_raw  ) )

txt_ref_len = len( txt_ref )
txt_trg_len = len( txt_trg )

In [4]:
# params 

padding = 2
gram_n = 3
sim_cut_off = 0.7

In [5]:
def nGram( text, n=3 ):
    length = len( text )
    return [ text[i:i+n] for i in range( length - n +1 ) ]

In [12]:
# text_ref
triGram_ref = nGram( txt_ref, n=gram_n )
triGram_ref_c = Counter( triGram_ref )
triGram_ref_s = triGram_ref_c.most_common()
triGram_ref_g = set( triGram_ref )

In [13]:
# text_ref
triGram_trg = nGram( txt_trg, n=gram_n )
triGram_trg_c = Counter( triGram_trg )
triGram_trg_s = triGram_trg_c.most_common()
triGram_trg_g = set( triGram_trg )

In [18]:
triGram_both = triGram_ref_g.intersection( triGram_trg_g )
triGram_ref_only = triGram_ref_g.difference( triGram_trg_g )
triGram_trg_only = triGram_trg_g.difference( triGram_ref_g )

print( len(triGram_both), len(triGram_ref_only), len(triGram_trg_only) )

20260 44366 491263


In [21]:
ref_mask = [0] * txt_ref_len
trg_mask = [0] * txt_trg_len

for gram in tqdm( triGram_both ) :
    for m in re.finditer(gram, txt_ref):
        ref_mask[ m.start():m.start()+gram_n ] = [1] * gram_n
    
    for m in re.finditer(gram, txt_trg):
        trg_mask[ m.start():m.start()+gram_n ] = [1] * gram_n


In [27]:
def get_ranges( mask, min_length=8 ):
    b, e = 0, 0
    rst = []
    for i, v in enumerate(mask):
        if v == mask[i-1]: continue
        if v > mask[i-1]: b = i
        if v < mask[i-1]: 
            e = i
            if e-b >= min_length:
                rst.append( (b, e ) )
    return rst

In [31]:
ref_ranges = get_ranges(ref_mask)
trg_ranges = get_ranges(trg_mask)
print( len(ref_ranges), len(trg_ranges ))

1552 1558


In [32]:
ref_ranges

[(36, 76),
 (77, 96),
 (98, 130),
 (132, 145),
 (146, 201),
 (202, 222),
 (223, 268),
 (269, 333),
 (335, 465),
 (466, 508),
 (509, 521),
 (522, 570),
 (571, 603),
 (646, 654),
 (685, 723),
 (724, 771),
 (772, 807),
 (808, 824),
 (827, 858),
 (859, 882),
 (883, 915),
 (920, 1199),
 (1206, 1219),
 (1368, 1452),
 (1488, 1500),
 (1505, 1513),
 (1559, 1571),
 (1671, 1709),
 (1725, 1740),
 (1741, 1768),
 (1777, 1862),
 (1864, 1888),
 (1895, 1903),
 (1907, 1915),
 (1928, 1956),
 (1986, 1996),
 (2024, 2077),
 (2095, 2109),
 (2119, 2127),
 (2149, 2157),
 (2172, 2192),
 (2203, 2216),
 (2233, 2312),
 (2313, 2332),
 (2334, 2395),
 (2396, 2417),
 (2437, 2445),
 (2464, 2472),
 (2519, 2538),
 (2568, 2580),
 (2603, 2611),
 (2655, 2667),
 (2674, 2698),
 (2701, 2727),
 (2732, 2758),
 (2784, 2829),
 (2831, 2842),
 (2908, 2916),
 (2918, 2927),
 (2928, 2946),
 (2951, 2970),
 (3022, 3031),
 (3032, 3044),
 (3079, 3103),
 (3137, 3156),
 (3196, 3207),
 (3308, 3322),
 (3334, 3346),
 (3388, 3408),
 (3442, 3466)

In [33]:
txt_ref[36:600]

'曰余聞上古之人春秋皆度百歲而動作不衰今時之人年半百而動作皆衰者時世異耶人將失之耶歧伯對曰上古之人其知道者法於陰陽和於術數食飮有節起居有常不妄作勞故能形與神俱而盡終其天年度百歲乃去今時之人不然也以酒爲漿以妄爲常醉以入房以欲竭其精以耗散其眞不知持滿不時御神務快其心逆於生樂起居無節故半百而衰也夫上古聖人之敎下也皆謂之虛邪賊風避之有時恬惔虛無眞氣從之精神內守病安從來是以志閑而少欲心安而不懼形勞而不倦氣從以順各從其欲皆得所願故美其食任其服樂其俗高下不相慕其民故曰朴是以嗜欲不能勞其目淫邪不能惑其心愚智賢不肖不懼於物故合於道所以能年皆度百歲而動作不衰者以其德全不危也帝曰人年老而無子者材力盡耶將天數然也歧伯曰女子七歲腎氣盛齒更髮長二七而天癸至任脈通太衝脈盛月事以時下故有子三七腎氣平均故眞牙生而長極四七筋骨堅髮長極身體盛壯五七陽明脈衰面始焦髮始墮六七三陽脈衰於上面皆焦髮始白七七任脈虛太衝脈衰少天癸竭地道不通故形壞而無子也丈夫八歲腎氣實髮長齒更二八腎氣盛天癸至精氣溢寫陰陽和故能有子三八腎氣平均筋骨勁强故眞牙生而長極四八筋骨隆盛肌肉滿壯五八腎氣衰髮墮齒槁六八陽氣衰竭於上面焦髮鬢頒白七八肝氣衰筋不能動天癸竭精少腎藏衰形體皆極八八則齒髮去腎者主水受五藏六府之精而藏之故五藏盛乃能寫今五藏皆衰筋骨解墮天癸盡矣故髮鬢白身體重行步不正而無子耳帝'

In [35]:
rst = []

for i_b, i_e in tqdm(ref_ranges):
    for j_b, j_e in trg_ranges:
        r = txt_ref[i_b:i_e]
        t = txt_trg[j_b:j_e]
        sim = SequenceMatcher(None,  r, t ).ratio()
        if sim <= sim_cut_off: continue
        rst.append( ( ( i_b, i_e ), ( j_b, j_e ) )  )

In [36]:
style = """
<style>
.match { 
    color: blue;
}
</style>

"""

In [37]:
handler = open("rst2.html", 'w', encoding="utf-8")
i = 1
border_size = 20
for ( i_b, i_e ), ( j_b, j_e)  in rst:    
    handler.write( "<div>" )
    handler.write( "\t<h2>{:d}</h2>\n".format(i) )
    handler.write( "\t<p class='ref'>{}<span class='match'>{}</span>{}</p>".format( txt_ref[i_b-border_size:i_b], txt_ref[i_b:i_e], txt_ref[i_e:i_e+border_size] ) )
    handler.write( "\t<p class='rtg'>{}<span class='match'>{}</span>{}</p>".format( txt_trg[j_b-border_size:j_b], txt_trg[j_b:j_e], txt_trg[j_e:j_e+border_size] ) )
    handler.write( "\n</div>" )
    handler.write( "<br><br>")
    i += 1

handler.write( style )
handler.close()